In [1]:
# 使用sys.path添加上级目录
import sys
import os
package_path = os.path.dirname(os.path.dirname(os.getcwd()))
file_path = os.path.join(package_path, "ch05", "06_user_interface")
print(file_path)
sys.path.append(file_path)

import torch
if torch.cuda.is_available():
   device = torch.device("cuda")
elif torch.backends.mps.is_available():
   device = torch.device("mps")
else:
   device = torch.device("cpu")

/Users/young/project/llmProject/LLMs-from-scratch-CN/ch05/06_user_interface


In [4]:
# ! pip3 install sentencepiece

In [5]:
from importlib.metadata import version

pkgs = [
    "huggingface_hub",
    "sentencepiece",
    "torch",
]
for p in pkgs:
    print(f"{p} version: {version(p)}")

huggingface_hub version: 0.29.2
sentencepiece version: 0.2.0
torch version: 2.6.0


# 1.逐步转换GPT模型的实现

## 1.1 替换LayerNorm为RMSNorm层

In [7]:
import torch
import torch.nn as nn

#####################################
# Chapter 4
#####################################

# class LayerNorm(nn.Module):
#     def __init__(self, emb_dim):
#         super().__init__()
#         self.eps = 1e-5
#         self.scale = nn.Parameter(torch.ones(emb_dim))
#         self.shift = nn.Parameter(torch.zeros(emb_dim))

#     def forward(self, x):
#         mean = x.mean(dim=-1, keepdim=True)
#         var = x.var(dim=-1, keepdim=True, unbiased=False)
#         norm_x = (x - mean) / torch.sqrt(var + self.eps)
#         return self.scale * norm_x + self.shift

class RMSNorm(nn.Module):
    def __init__(self, emb_dim, eps=1e-5):
        super().__init__()
        self.eps = eps
        self.emb_dim = emb_dim
        self.weight = nn.Parameter(torch.ones(emb_dim)).float()

    def forward(self, x):
        means = x.pow(2).mean(dim=-1, keepdim=True)
        x_normed = x * torch.rsqrt(means + self.eps)
        return (x_normed * self.weight).to(dtype=x.dtype)

In [10]:
torch.manual_seed(123)

# [batch_size, seq_len, emb_dim]
example_batch = torch.randn(2, 3, 4)

rms_norm = RMSNorm(emb_dim=example_batch.shape[-1])
rmsnorm_pytorch = torch.nn.RMSNorm(example_batch.shape[-1], eps=1e-5)

print(rms_norm(example_batch))
print("*" * 30)
print(rmsnorm_pytorch(example_batch))


assert torch.allclose(rms_norm(example_batch), rmsnorm_pytorch(example_batch))

tensor([[[ 0.8834, -0.4655, -0.7948, -1.5398],
         [ 0.8053,  1.5254, -0.5074, -0.8759],
         [-0.1619, -0.4049,  0.7016, -1.8214]],

        [[ 1.3348, -0.1232,  0.9638, -1.1288],
         [-0.8511, -1.5285, -0.4420,  0.8624],
         [ 0.9398,  0.2082,  1.7241, -0.3180]]], grad_fn=<MulBackward0>)
******************************
tensor([[[ 0.8834, -0.4655, -0.7948, -1.5398],
         [ 0.8053,  1.5254, -0.5074, -0.8759],
         [-0.1619, -0.4049,  0.7016, -1.8214]],

        [[ 1.3348, -0.1232,  0.9638, -1.1288],
         [-0.8511, -1.5285, -0.4420,  0.8624],
         [ 0.9398,  0.2082,  1.7241, -0.3180]]], grad_fn=<MulBackward0>)


## 1.2 用SiLU代替GELU作激活函数

In [11]:
#####################################
# Chapter 4
#####################################

# class GELU(nn.Module):
#     def __init__(self):
#         super().__init__()

#     def forward(self, x):
#         return 0.5 * x * (1 + torch.tanh(
#             torch.sqrt(torch.tensor(2.0 / torch.pi)) *
#             (x + 0.044715 * torch.pow(x, 3))
#         ))

class SiLU(nn.Module):
    def __init__(self):
        super(SiLU, self).__init__()

    def forward(self, x):
        return x * torch.sigmoid(x)

In [12]:
silu = SiLU()

assert torch.allclose(silu(example_batch), torch.nn.functional.silu(example_batch))

## 1.3 更新FeedForward模块

In [ ]:
#####################################
# Chapter 4
#####################################
# class FeedForward(nn.Module):
#     def __init__(self, cfg):
#         super().__init__()
#         self.layers = nn.Sequential(
#             nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
#             GELU(),
#             nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),
#         )

#     def forward(self, x):
#         return self.layers(x)

class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.fc1 = nn.Linear(cfg["emb_dim"], cfg["hidden_dim"], dtype=cfg["dtype"], bias=False)
        self.fc2 = nn.Linear(cfg["emb_dim"], cfg["hidden_dim"], dtype=cfg["dtype"], bias=False)
        self.fc3 = nn.Linear(cfg["hidden_dim"], cfg["emb_dim"], dtype=cfg["dtype"], bias=False)
        self.silu = SiLU()
    
    def forward(self, x):
        x_fc1 = self.fc1(x)
        x_fc2 = self.fc2(x)
        x = self.silu(x_fc1) * x_fc2
        return self.fc3(x)

## 1.4 实现RoPE旋转位置编码

In [ ]:
# 预计算RoPE参数
def precompute_rope_params(head_dim, theta_base=10_000, context_length=1024):
    assert head_dim % 2 == 2, "Embedding dimension must be even"

    # 计算频率的倒数
    inv_freq = 1.0 / (theta_base **(torch.arange(0, head_dim, 2)[: (head_dim // 2)].float() / head_dim))

    # 生成位置索引
    positions = torch.arange(context_length)

    # 计算旋转角度
    # 通过维度扩展+广播实现
    # positions: [context_length] -> [context_length, 1]
    # inv_freq: [head_dim // 2] -> [1, head_dim // 2]
    # angles: [context_length, head_dim // 2]
    angles = positions[:, None] * inv_freq[None, :]

    # 计算正弦和余弦值
    sin = torch.sin(angles)
    cos = torch.cos(angles)

    return cos, sin

# 计算RoPE
def compute_rope(x, cos, sin):
    # x: (batch_size, num_heads, seq_len, head_dim)
    batch_size, num_heads, seq_len, head_dim = x.shape
    assert head_dim % 2 ==0, "Head dimension must be even"

    # 将x划分为两部分，作为复数的实部和虚部
    x1 = x[..., : head_dim // 2]
    x2 = x[..., head_dim // 2 :]

    # shape: (1, 1, seq_len, head_dim)
    cos = cos[:seq_len, :].unsqueeze(0).unsqueeze(0)
    sin = sin[:seq_len, :].unsqueeze(0).unsqueeze(0)

    # 应用旋转变换
    rotated = torch.cat((-x2, x1), dim=-1)
    x_rotated = (x * cos) + (rotated * sin)

    return x_rotated.to(dtype=x.dtype)

In [ ]:
# 示例：将rope应用于q和k
batch_size = 2
context_len = 5
num_heads = 4
head_dim = 16

# 初始化
cos, sin = precompute_rope_params(head_dim=head_dim, context_length=context_len)

# 构建测试用q和k
torch.manual_seed(123)
queries = torch.randn(batch_size, num_heads, context_len, head_dim)
keys = torch.rnadn(batch_size, num_heads, context_len, head_dim)

# 应用rope到pos emb
queries_rot = compute_rope(queries, cos, sin)
keys_rot = compute_rope(keys, cos, sin)


## 1.5 在`MultiHeadAttention`模块中添加RoPE

In [ ]:
# 基于Chapter 3的多头注意进行修改
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, num_heads, dtype=None):
        super().__init__()
        assert d_out % num_heads == 0, "d_out must be divisible by n_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads

        ################################### NEW ###################################
        # 不同点：Set bias=False and dtype=dtype for all linear layers below
        ########################################################################### 
        self.W_query = nn.Linear(d_in, d_out, bias=False, dtype=dtype)
        self.W_key = nn.Linear(d_in, d_out, bias=False, dtype=dtype)
        self.W_value = nn.Linear(d_in, d_out, bias=False, dtype=dtype)
        self.out_proj = nn.Linear(d_out, d_out, bias=False, dtype=dtype)
        ################################### NEW ###################################
        # 不同点：去除dropout
        ########################################################################### 
        # self.dropout = nn.Dropout(dropout)
        self.register_buffer("mask", torch.triu(torch.ones(context_length, context_length), diagonal=1))

        ################################### NEW ###################################
        # 不同点：使用rope
        ########################################################################### 
        cos, sin = precompute_rope_params(head_dim=self.head_dim, context_length=context_length)
        self.register_buffer("cos", cos)
        self.register_buffer("sin", sin)
    
    def forward(self, x):
        b, num_tokens, d_in = x.shape

        key
